In [1]:
import pandas as pd
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
import nltk
from nltk.corpus import stopwords
import random

import warnings; warnings.simplefilter('ignore')

In [2]:
data=pd.read_csv('result_final.csv')

In [3]:
data=data.rename(columns={'text':'sub-title'})
data.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'link', 'sub-title', 'title', 'date',
       'keywords', 'summary', 'title_summary'],
      dtype='object')

In [4]:
data=data.drop(['Unnamed: 0', 'Unnamed: 0.1','title_summary'],axis=1)

In [5]:
data.insert(0,'id',range(0,data.shape[0]))
data

,id,link,sub-title,title,date,keywords,summary
0,0,http://techcrunch.com/2020/09/07/vodafone-idea...,"Vodafone Idea, one of the largest telecom oper...",Indian telecom giant Vodafone Idea rebrands as...,2020-09-07 00:00:00,"['rebrands', 'idea', 'vi', 'giant', 'brand', '...","Vodafone Idea, one of the largest telecom oper..."
1,1,http://techcrunch.com/2020/09/16/facebook-addr...,"At the beginning of the previous decade, Faceb...",Facebook addresses political controversy in In...,2020-09-16 00:00:00,"['opportunities', 'whatsapp', 'controversy', '...",Politicians in the country today heavily rely ...
2,2,http://techcrunch.com/2020/09/14/youtube-launc...,"As TikTok’s fate in the U.S. remains murky, Yo...","YouTube launches its TikTok rival, YouTube Sho...",2020-09-14 00:00:00,"['rival', 'video', 'feature', 'access', 'youtu...","As TikTok’s fate in the U.S. remains murky, Yo..."
3,3,http://techcrunch.com/2020/09/09/groww-an-inve...,Even as more than 150 million people are using...,"Groww, an investment app for millennials in In...",2020-09-09 00:00:00,"['world', 'yc', 'continuity', 'stocks', 'groww...","YC Continuity, the growth-stage investment fun..."
4,4,http://techcrunch.com/2020/09/15/lanzatech-is-...,As part of the continuing global rollout of La...,LanzaTech is developing a small-scale waste bi...,2020-09-15 00:00:00,"['production', 'distributed', 'developing', 's...",As part of the continuing global rollout of La...
...,...,...,...,...,...,...,...
2185,2185,https://uk.reuters.com/article/uk-brazil-econo...,BRASILIA (Reuters) - Paulo Guedes has faced hi...,Brazil's Guedes finds influence waning as Bols...,2020-09-16 18:18:18+00:00,"['bolsonaro', 'guedes', 'reins', 'economic', '...",FILE PHOTO: Brazil's Economy Minister Paulo Gu...
2186,2186,https://www.reuters.com/article/us-britain-boe...,LONDON (Reuters) - The Bank of England is expe...,Bank of England gears up for next stimulus push,2020-09-16 23:07:30+00:00,"['push', 'bank', 'england', 'zero', 'signal', ...",LONDON (Reuters) - The Bank of England is expe...
2187,2187,https://www.reuters.com/article/syria-crisis-f...,AMMAN (Reuters) - Syria is experiencing worsen...,Syria says U.S. sanctions behind acute fuel cr...,2020-09-16 23:25:09+00:00,"['shortages', 'imports', 'gasoline', 'shipment...",AMMAN (Reuters) - Syria is experiencing worsen...
2188,2188,https://af.reuters.com/article/uk-iran-usa-wor...,Curiosity high for TV's anything-can-happen vi...,"Business & Financial News, U.S & International...",NaN,"['international', 'tvs', 'sundays', 'united', ...",Curiosity high for TV's anything-can-happen vi...


In [6]:
data=data.set_index('id')
data

,link,sub-title,title,date,keywords,summary
id,,,,,,
0,http://techcrunch.com/2020/09/07/vodafone-idea...,"Vodafone Idea, one of the largest telecom oper...",Indian telecom giant Vodafone Idea rebrands as...,2020-09-07 00:00:00,"['rebrands', 'idea', 'vi', 'giant', 'brand', '...","Vodafone Idea, one of the largest telecom oper..."
1,http://techcrunch.com/2020/09/16/facebook-addr...,"At the beginning of the previous decade, Faceb...",Facebook addresses political controversy in In...,2020-09-16 00:00:00,"['opportunities', 'whatsapp', 'controversy', '...",Politicians in the country today heavily rely ...
2,http://techcrunch.com/2020/09/14/youtube-launc...,"As TikTok’s fate in the U.S. remains murky, Yo...","YouTube launches its TikTok rival, YouTube Sho...",2020-09-14 00:00:00,"['rival', 'video', 'feature', 'access', 'youtu...","As TikTok’s fate in the U.S. remains murky, Yo..."
3,http://techcrunch.com/2020/09/09/groww-an-inve...,Even as more than 150 million people are using...,"Groww, an investment app for millennials in In...",2020-09-09 00:00:00,"['world', 'yc', 'continuity', 'stocks', 'groww...","YC Continuity, the growth-stage investment fun..."
4,http://techcrunch.com/2020/09/15/lanzatech-is-...,As part of the continuing global rollout of La...,LanzaTech is developing a small-scale waste bi...,2020-09-15 00:00:00,"['production', 'distributed', 'developing', 's...",As part of the continuing global rollout of La...
...,...,...,...,...,...,...
2185,https://uk.reuters.com/article/uk-brazil-econo...,BRASILIA (Reuters) - Paulo Guedes has faced hi...,Brazil's Guedes finds influence waning as Bols...,2020-09-16 18:18:18+00:00,"['bolsonaro', 'guedes', 'reins', 'economic', '...",FILE PHOTO: Brazil's Economy Minister Paulo Gu...
2186,https://www.reuters.com/article/us-britain-boe...,LONDON (Reuters) - The Bank of England is expe...,Bank of England gears up for next stimulus push,2020-09-16 23:07:30+00:00,"['push', 'bank', 'england', 'zero', 'signal', ...",LONDON (Reuters) - The Bank of England is expe...
2187,https://www.reuters.com/article/syria-crisis-f...,AMMAN (Reuters) - Syria is experiencing worsen...,Syria says U.S. sanctions behind acute fuel cr...,2020-09-16 23:25:09+00:00,"['shortages', 'imports', 'gasoline', 'shipment...",AMMAN (Reuters) - Syria is experiencing worsen...


In [7]:
data.isnull().sum()

link           0
sub-title      6
title          3
date         499
keywords       3
summary        6
dtype: int64

In [8]:
data=data.dropna()

In [9]:
#a=data['sub-title'].str.split()
#print(a)
data.shape

(1689, 6)

In [10]:
#data wrangling 
#we will remove all the stopwords those are unnecessary from the text. The words like is,a,the,etc
def remove_stopwords(news):
    news=news.split()
    news_without_stopwords=[]
    en_stops = set(stopwords.words('english'))
    for word in news: 
        if word not in en_stops:
            news_without_stopwords.append(word)
    news=news_without_stopwords                      #news will be changed to string without unnceccessary words.
    return news

#Using the above created functions for getting clean data (without unwanted words and in a single format)
data['changed_summary'] = data['summary'].str.lower()
data['changed_summary'] =  data.changed_summary.apply(func = remove_stopwords)
data['changed_summary']=[" ".join(review) for review in data['changed_summary'].values]

In [11]:
for i in range(len(data)):
    data.iloc[i,4] = " ".join(eval(data.iloc[i,4]))

In [12]:
data['keywords']

id
0       rebrands idea vi giant brand largest billion t...
1       opportunities whatsapp controversy role politi...
2       rival video feature access youtube shorts tech...
3       world yc continuity stocks groww techcrunch mi...
4       production distributed developing scale biomas...
                              ...                        
2182    progress europes fuelfree electricity renewabl...
2185    bolsonaro guedes reins economic finds fiscal b...
2186    push bank england zero signal central rates ob...
2187    shortages imports gasoline shipments major syr...
2189    copper sessions metalslme seven dollar active ...
Name: keywords, Length: 1689, dtype: object

In [13]:
data['changed_summary']

id
0       vodafone idea, one largest telecom operators i...
1       politicians country today heavily rely faceboo...
2       tiktok’s fate u.s. remains murky, youtube toda...
3       yc continuity, growth-stage investment fund co...
4       part continuing global rollout lanzatech’s tec...
                              ...                        
2182    reuters/gustau nacarinolondon (reuters) - much...
2185    file photo: brazil's economy minister paulo gu...
2186    london (reuters) - bank england expected signa...
2187    amman (reuters) - syria experiencing worsening...
2189    melbourne, sept 17 (reuters) - london copper f...
Name: changed_summary, Length: 1689, dtype: object

In [14]:
a=data['changed_summary']
a[0]

'vodafone idea, one largest telecom operators india, rebranded ‘vi’ looks “fresh start” three years british telecom giant vodafone group’s india business billionaire kumar mangalam birla’s idea cellular merged country. that’s believe perfect time launch vi, one company provides strength vodafone india idea,” vodafone group ceo nick read said virtual conference monday. vodafone idea, largest telecom operator country 400 million subscribers, lost 100 million subscribers recent years new comer jio platforms recent years. india’s richest man mukesh ambani’s telecom venture become largest telecom operator nearly 400 million subscribers cut-rate mobile data tariffs. “india second largest telecom market largest data consumer, globally.'

In [15]:
tfidf = TfidfVectorizer(analyzer='word',norm=None,use_idf=True,ngram_range=(1,3),smooth_idf=True)
matrix = tfidf.fit_transform(data['keywords'])

In [16]:
sorted(" ".join(data['keywords']).split(" "))

['0123',
 '0124',
 '02',
 '025',
 '032',
 '04',
 '05',
 '07apr',
 '07apr',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '100',
 '1000',
 '1000',
 '10000',
 '100000',
 '1000000',
 '1000000',
 '1005',
 '1005',
 '1005',
 '1005',
 '1005',
 '100m',
 '100m',
 '100m',
 '100m',
 '100m',
 '100th',
 '1015',
 '10273',
 '1030',
 '1030',
 '10b',
 '11',
 '11',
 '11',
 '11',
 '11',
 '11',
 '11',
 '11',
 '1100',
 '1100',
 '112',
 '112week',
 '112year',
 '112year',
 '112yr',
 '112yr',
 '1135',
 '1135',
 '118',
 '118',
 '118',
 '118m',
 '118m',
 '118m',
 '118m',
 '118m',
 '11course',
 '12',
 '12',
 '12',
 '12',
 '12',
 '1200',
 '1200',
 '12m',
 '12m',
 '12m',
 '13',
 '13',
 '1300',
 '1300',
 '1300',
 '1300',
 '137',
 '137',
 '13course',
 '14',
 '14',
 '14',
 '14',
 '14',
 '14',
 '14',
 '1415',
 '1415',
 '1415',
 '1415',
 '1415',
 '1415',
 '1415',
 '1426',
 '144hz',
 '14th',
 '14th',
 '14th',
 '15',
 '15',
 '15',
 '15',
 '15',
 '15',
 '15',
 '15',
 '15',
 '15',
 '156',
 '16',
 '16',
 '16',
 '

In [17]:
tfidf.get_feature_names()

['0123',
 '0123 dlr',
 '0123 dlr bank',
 '0124',
 '0124 15',
 '02',
 '02 stimulus',
 '02 stimulus standard',
 '025',
 '025 funding',
 '025 funding update',
 '032',
 '032 shares',
 '04',
 '04 aug',
 '04 aug unchangedthe',
 '05',
 '05 jgb',
 '05 jgb fall',
 '07apr',
 '07apr bids',
 '07apr bids securities',
 '100',
 '100 100m',
 '100 100m company',
 '100 2020',
 '100 2020 startup',
 '100 indian',
 '100 indian banned',
 '100 popup',
 '100 popup carbon',
 '100 revenue',
 '100 revenue million',
 '100 techcrunch',
 '100 techcrunch digital',
 '100 techcrunch tiktok',
 '1000',
 '1000 perks',
 '1000 perks crunch',
 '1000 york',
 '1000 york shootings',
 '10000',
 '10000 surpassed',
 '10000 surpassed weeks',
 '100000',
 '100000 evacuate',
 '100000 evacuate air',
 '1000000',
 '1000000 07apr',
 '1000000 07apr bids',
 '1005',
 '1005 24',
 '1005 24 mth',
 '1005 mth',
 '1005 mth 1300',
 '1005 mth 419',
 '1005 mth 432',
 '100m',
 '100m company',
 '100m company hits',
 '100m europe',
 '100m europe ways',

In [18]:
cosine_similarities = cosine_similarity(matrix,matrix)

In [19]:
def filter_function(x):
    if(x[1]>1000):
        return False
    return True

In [20]:
def recommendation(id):
    #get similarity values with other articles
    similarity_score =  [(el[0],round(el[1],3)) for el in list(enumerate(cosine_similarities[id]))]
    similarity_score = sorted(similarity_score, key=lambda x: x[1], reverse=True)
    similarity_score = list(filter(filter_function,similarity_score))
    sim2 = []
    y = [j[1] for j in similarity_score]
    for i in range(len(similarity_score)):
        x = similarity_score[i][1]
        if(x not in y[i+1:]):
            sim2.append(similarity_score[i])
    similarity_score = random.sample(sim2[:10],5)
    
    print("News-->  " + data['title'].iloc[id])
    print(" ")
    news = [i[0] for i in similarity_score]
    for i in range(len(news)):
        print("Recomendation "+ str(i+1)+" : "+
              data['title'].iloc[news[i]])
        print("Link -->"+ data['link'].iloc[news[i]])
        print(" score --> "+ str(similarity_score[i][1]))
        print()

In [21]:
recommendation(2)

News-->  YouTube launches its TikTok rival, YouTube Shorts, initially in India – TechCrunch
 
Recomendation 1 : 'Oracle just completely saved the day': TikTok users react to looming deal
Link -->https://www.cnn.com/2020/09/14/tech/tiktok-oracle-user-reaction/index.html
 score --> 0.048

Recomendation 2 : The pandemic backed YouTube into a moderation corner. It has the numbers to prove it.
Link -->https://mashable.com/article/youtube-content-moderation-automated-systems/
 score --> 0.033

Recomendation 3 : Oracle boots out Microsoft and wins bid for TikTok, reports say – TechCrunch
Link -->http://techcrunch.com/2020/09/13/oracle-wins-bid-to-buy-tiktok/
 score --> 0.034

Recomendation 4 : Walmart-exclusive TrillerTok will run on Azure, or Oracle, or something – TechCrunch
Link -->http://techcrunch.com/2020/08/29/what-is-going-on-anymore/
 score --> 0.05

Recomendation 5 : YouTube officially announces TikTok competitor, YouTube Shorts
Link -->https://mashable.com/article/youtube-announces

In [22]:
def filter_function_2(x):
    if(x[1]>0.4):
        return False
    return True

In [23]:
def recommendation2(id):
    #get similarity values with other articles
    similarity1 = [(el[0],round(el[1],3)) for el in list(enumerate(cosine_similarities[id]))]
    similarity2 = [(el[0],round(el[1],3)) for el in list(enumerate(cosine_similarities[id]))]
    similarity_score = [(x[0],x[1]+y[1]) for x,y in zip(similarity1,similarity2)]
    similarity_score = sorted(similarity_score, key=lambda x: x[1], reverse=True)
    similarity_score = list(filter(filter_function_2,similarity_score))
    sim2 = []
    y = [j[1] for j in similarity_score]
    for i in range(len(similarity_score)):
        x = similarity_score[i][1]
        if(x not in y[i+1:]):
            sim2.append(similarity_score[i])
    similarity_score = random.sample(sim2[:10],5)
    
    print("News-->  " + data['title'].iloc[id])
    print(" ")
    news_indices = [i[0] for i in similarity_score]
    for i in range(len(news_indices)):
        print("Recomendation "+ str(i+1)+" : "+
              data['title'].iloc[news_indices[i]])
        print("Link -->"+ data['link'].iloc[news_indices[i]])
        print(" score --> "+ str(similarity_score[i][1]))
        print()

In [24]:
recommendation2(2)

News-->  YouTube launches its TikTok rival, YouTube Shorts, initially in India – TechCrunch
 
Recomendation 1 : 'Oracle just completely saved the day': TikTok users react to looming deal
Link -->https://www.cnn.com/2020/09/14/tech/tiktok-oracle-user-reaction/index.html
 score --> 0.096

Recomendation 2 : TikTok hits 100M users in Europe as the clock ticks on its US business – TechCrunch
Link -->http://techcrunch.com/2020/09/14/tiktok-hits-100m-users-in-europe-as-the-clock-ticks-on-its-us-business/
 score --> 0.086

Recomendation 3 : YouTube officially announces TikTok competitor, YouTube Shorts
Link -->https://mashable.com/article/youtube-announces-youtube-shorts-tiktok-competitor/
 score --> 0.344

Recomendation 4 : Charli D'Amelio joins Triller, TikTok's rival app
Link -->https://mashable.com/article/charli-damelio-triller-tiktok-rival/
 score --> 0.064

Recomendation 5 : Walmart-exclusive TrillerTok will run on Azure, or Oracle, or something – TechCrunch
Link -->http://techcrunch.co